### Example of using Bedrock for video generation

* Invoke model to generate video using **start_async_invoke**
* Invoke model to generate text using **invoke_model**
* Invoke model to generate streaming real time text using **invoke_model_with_response_stream**

https://aws.amazon.com/blogs/aws/amazon-nova-reel-1-1-featuring-up-to-2-minutes-multi-shot-videos/



In [9]:
# in same terminal as notebook launch, export AWS_PROFILE first

!aws sts get-caller-identity

{
    "UserId": "AIDAQQTN2DF4X2R2TT27O",
    "Account": "035663780217",
    "Arn": "arn:aws:iam::035663780217:user/devs"
}


In [5]:
import json
import random
import boto3

bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
bedrock = boto3.client(service_name="bedrock", region_name="us-east-1")

s3 = boto3.client("s3")
model_id = "amazon.nova-reel-v1:0"

prompt = "A person dancing on a mountain"

seed = random.randint(0, 2147483646)

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {"text": prompt},
    "videoGenerationConfig": {
        "fps": 24,
        "durationSeconds": 6,
        "dimension": "1280x720",
        "seed": seed,
    }
}

output_config = {
    "s3OutputDataConfig": {
        "s3Uri": "s3://genai-exercise-cky/video/"
    }
}

response = bedrock_runtime.start_async_invoke(
    modelId=model_id,
    modelInput=model_input,
    outputDataConfig=output_config
)


invocation_arn = response["invocationArn"]
print("Job submitted")
print(f"Invocation ARN: {invocation_arn}")

Job submitted
Invocation ARN: arn:aws:bedrock:us-east-1:035663780217:async-invoke/cf8hg0qqwdzm


In [6]:
job_status = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
print(f"Current status: {job_status['status']}")

Current status: Completed


In [7]:
import json
import boto3

MODEL_ID = "amazon.nova-micro-v1:0"
bedrock = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

response = bedrock.invoke_model(
    modelId=MODEL_ID,
    guardrailVersion="DRAFT",
    body=json.dumps({
        "schemaVersion": "messages-v1",
        "messages": [{"role": "user", "content": [{"text": "Rewrite this sentence for me in a formal tone: You are very good at your job."}]}],
        "inferenceConfig": {"maxTokens": 500, "topK": 20, "temperature": 0.7}
    })
)

response["body"].read()



    

b'{"output":{"message":{"content":[{"text":"Your proficiency in your professional responsibilities is commendable."}],"role":"assistant"}},"stopReason":"end_turn","usage":{"inputTokens":19,"outputTokens":11,"totalTokens":30,"cacheReadInputTokenCount":0,"cacheWriteInputTokenCount":0}}'

In [8]:
# example of using streaming text
import json
import boto3

bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')
model_id = 'amazon.nova-micro-v1:0'

payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "Tell me what type of dances people do."
                }
            ]
        }
    ]
}

payload_json = json.dumps(payload)

response = bedrock.invoke_model_with_response_stream(
    modelId=model_id,
    body=payload_json,
    contentType='application/json',
    accept='application/json'
)

stream = response['body']
full_text = ""

print("Receiving response stream:")
for event in stream:
    chunk = event['chunk']
    chunk_str = chunk.get('bytes', b'').decode('utf-8')

    try:
        json_chunk = json.loads(chunk_str)

        if "contentBlockDelta" in json_chunk:
            delta = json_chunk["contentBlockDelta"]["delta"]
            text = delta.get("text", "")
            full_text += text
            print(text, end='') 

    except json.JSONDecodeError:
        continue

Receiving response stream:
There are numerous types of dances practiced around the world, each with its own unique style, history, and cultural significance. Here's an overview of some popular categories and specific dances within those categories:

### 1. **Traditional/Folk Dances**
These dances often reflect the cultural heritage of a particular region or community.

- **African Dances**: Include styles like Zulu dance, highlife, and Djembe.
- **Latin American Dances**: Salsa, Bachata, Merengue, and Tango.
- **European Dances**: Flamenco from Spain, Irish stepdance, and Polish Mazurka.
- **Asian Dances**: Bollywood dances, Indian classical dances like Bharatanatyam and Kathak, and Japanese Kagura.

### 2. **Ballroom Dances**
These are formal partner dances that are usually performed on a smooth wooden floor in a ballroom setting.

- **Waltz**: A smooth, triple-time dance.
- **Tango**: An intense, dramatic dance with sharp, staccato movements.
- **Foxtrot**: A smooth dance with glidin